In [41]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import json
import random

### Data Preparation

In [38]:
def get_dataset(directory):
    feature = []
    label = []
    for foldername in os.listdir(directory):
        folder = os.path.join(directory, foldername)
        if os.path.isdir(folder):
            files = os.listdir(folder)
            for filename in files:
                rel_path = os.path.join(directory, foldername, filename)
                temp_label = filename.split('.')[0].split('_')[0]
                if 'a' in temp_label:
                    label = np.append(label,'alcoholic')
                else:
                    label = np.append(label,'control')
                
                df_data = np.loadtxt(rel_path, delimiter=",")
                feature.append(df_data.T)
    return feature, label

### Model Definition

In [4]:
def create_model():
    input_layer = keras.Input(shape=(512, 1))

    x = layers.Conv1D(filters=8, kernel_size=8, strides=2, activation="relu")(input_layer)
    x = layers.MaxPooling1D(pool_size=2)(x)

    x = layers.Conv1D(filters=4, kernel_size=4, strides=2, activation="relu")(x)
    x = layers.MaxPooling1D(pool_size=2)(x)

    x = layers.Flatten()(x)

    x = layers.Dense(4096, activation="relu")(x)

    x = layers.Dense(2048, activation="relu")(x)

    output_layer = layers.Dense(2, activation="softmax")(x)

    return keras.Model(inputs=input_layer, outputs=output_layer)

### Main Program

In [55]:
# loading extracted feature & label
x_test, y_test = get_dataset('../smni_cmi_test_feature_256')
# x_train, y_train = get_dataset('../Features/smni_cmi_train_feature_256')

In [57]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
train_batches = train_dataset.shuffle(len(y_test)).batch(32)
for element in train_batches:
  print(element)

(<tf.Tensor: shape=(32, 366), dtype=float64, numpy=
array([[1.72742654e+04, 5.20588045e-01, 2.27958245e+01, ...,
        6.02018293e+01, 9.62798816e+02, 4.94962289e+02],
       [5.08678385e+05, 4.89681546e+00, 2.73535930e+02, ...,
        6.13994625e+00, 1.14545673e+01, 7.33243441e+01],
       [3.24388602e+05, 1.05832879e+01, 1.69654016e+02, ...,
        1.08069646e+01, 5.05306683e+01, 1.38387241e+01],
       ...,
       [5.08215236e+02, 3.83525475e-01, 1.33506964e+01, ...,
        2.79302906e+00, 1.11243987e+01, 7.60098893e+00],
       [5.13991472e+05, 8.18089550e+00, 2.08255330e+02, ...,
        8.85093263e-01, 4.15916772e+00, 1.79109817e+01],
       [4.56426143e+04, 4.30988369e+00, 1.11825830e+02, ...,
        1.14037030e+01, 3.65826223e+01, 1.27556581e+02]])>, <tf.Tensor: shape=(32,), dtype=string, numpy=
array([b'control', b'alcoholic', b'alcoholic', b'control', b'alcoholic',
       b'alcoholic', b'control', b'alcoholic', b'alcoholic', b'control',
       b'alcoholic', b'alcoholic'